In [1]:
from kaggle_secrets import UserSecretsClient
secret_label = "Token"
token = UserSecretsClient().get_secret(secret_label)

! git clone https://{token}@github.com/CarloSgaravatti/Recsys_Challenge_2023.git

Cloning into 'Recsys_Challenge_2023'...
remote: Enumerating objects: 2734, done.
remote: Counting objects: 100% (852/852), done.
remote: Compressing objects: 100% (152/152), done.
remote: Total 2734 (delta 731), reused 804 (delta 692), pack-reused 1882
Receiving objects: 100% (2734/2734), 398.83 MiB | 30.01 MiB/s, done.
Resolving deltas: 100% (2183/2183), done.


In [2]:
! pip install PyGithub requests

  Obtaining dependency information for PyGithub from https://files.pythonhosted.org/packages/be/04/810d131be173cba445d3658a45512b2b2b3d0960d52c4a300d6ec5e00f52/PyGithub-2.1.1-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 15.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.9/302.9 kB 21.2 MB/s eta 0:00:00


In [3]:
! cd /kaggle/working/Recsys_Challenge_2023 && python run_compile_all_cython.py

run_compile_all_cython: Found 10 Cython files in 4 folders...
run_compile_all_cython: All files will be compiled using your current python environment: '/opt/conda/bin/python'
Compiling [1/10]: MatrixFactorizationImpressions_Cython_Epoch.pyx... 
In file included from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarraytypes.h:1940,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/ndarrayobject.h:12,
                 from /opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/arrayobject.h:5,
                 from MatrixFactorizationImpressions_Cython_Epoch.c:1109:
/opt/conda/lib/python3.10/site-packages/numpy/core/include/numpy/npy_1_7_deprecated_api.h:17:2: warning: #warning "Using deprecated NumPy API, disable it with " "#define NPY_NO_DEPRECATED_API NPY_1_7_API_VERSION" [-Wcpp]
   17 | #warning "Using deprecated NumPy API, disable it with " \
      |  ^~~~~~~
MatrixFactorizationImpressions_Cython_Epoch.c: In functio

In [4]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import json
import random
import tqdm
import gc
import time
import os
import shutil
import optuna
import scipy.sparse as sps

for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv
/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv
/kaggle/input/training-tuned-models/__results__.html
/kaggle/input/training-tuned-models/ease_r_all.zip
/kaggle/input/training-tuned-models/ease_r_train_validation.zip
/kaggle/input/training-tuned-models/__notebook__.ipynb
/kaggle/input/training-tuned-models/ials_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/ials_train_validation.zip
/kaggle/input/training-tuned-models/ease_r_xgboost_train_validation.zip
/kaggle/input/training-tuned-models/__output__.json
/kaggle/input/training-tuned-models/ials_all.zip
/kaggle/input/training-tuned-models/custom.css
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/run_all_algorithms.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/cython_example_FunkSVD.py
/kaggle/input/training-tuned-models/Recsys_Challenge_2023/LICENSE
/kaggle/input/training-tuned-models/Recsys_

In [5]:
%cd /kaggle/working/Recsys_Challenge_2023

/kaggle/working/Recsys_Challenge_2023


In [6]:
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender, SLIMElasticNetRecommender
from Evaluation.Evaluator import EvaluatorHoldout
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender, PureSVDItemRecommender
from Recommenders.KNN.ItemKNNCFRecommender import ItemKNNCFRecommender
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNCustomSimilarityRecommender import ItemKNNCustomSimilarityRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.SLIM.SLIM_BPR_Python import SLIM_BPR_Python
from Recommenders.MatrixFactorization.NMFRecommender import NMFRecommender
from Recommenders.BaseRecommender import BaseRecommender

from Recommenders.Hybrid.LinearCombinationRecommender import LinearCombinationRecommender
from Data_manager.CrossValidationSplitter import CrossValidationSplitter

np.int = int
np.random.seed(42)
np.bool = bool
np.float = float

# Configurations

In [7]:
from github import Github
from github import Auth

auth = Auth.Token(token)
g = Github(auth=auth)

repo = None
for r in g.get_user().get_repos():
    if r.name == 'Recsys_Challenge_2023':
        repo = r
        print('Repository found')

Repository found


In [8]:
def upload_file(filepath_kaggle, filepath_github, commit_message):
    try:
        contents = repo.get_contents(filepath_github)
        with open(filepath_kaggle, "rb") as file:
            repo.update_file(contents.path, commit_message, file.read(), contents.sha)
    except Exception:
        with open(filepath_kaggle, "rb") as file:
            repo.create_file(filepath_github, commit_message, file.read())

In [9]:
config = {
    'tune_parameters': True,
    'database_path': '/kaggle/working/tuning_hybrid.db',
    'copy_prev_tuning_db': True,
    'copy_prev_best_params': True,
    'save_github': True
}

# Tuning

In [10]:
train_data = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_train.csv')
user_ids_test = pd.read_csv('/kaggle/input/recommender-system-2023-challenge-polimi/data_target_users_test.csv')

In [11]:
URM_all = sps.csr_matrix((train_data.data.values,
                          ((train_data.row - 1).values, (train_data.col - 1).values)),
                        shape=(train_data.row.max(), train_data.col.max()))
URM_all

<13024x22347 sparse matrix of type '<class 'numpy.float64'>'
	with 478730 stored elements in Compressed Sparse Row format>

In [12]:
URM_train, URM_test_complete = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.8)

In [13]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_item.json', 'r') as params_file:
    params_item_knn = json.load(params_file)
params_item_knn

{'topK': 14,
 'shrink': 25,
 'similarity': 'tversky',
 'tversky_alpha': 0.18163095712123836,
 'tversky_beta': 1.8529812241698465}

In [14]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_RP3.json', 'r') as params_file:
    params_rp3 = json.load(params_file)
params_rp3

{'alpha': 0.3408516511081212, 'beta': 0.1447198697205665, 'topK': 32}

In [15]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_alpha.json', 'r') as params_file:
    params_p3alpha = json.load(params_file)
params_p3alpha

{'normalize_similarity': 'True',
 'implicit': 'True',
 'alpha': 0.29354071531607107,
 'topK': 49}

In [16]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_knn_user.json', 'r') as params_file:
    params_user_knn = json.load(params_file)
params_user_knn

{'topK': 498,
 'shrink': 0,
 'similarity': 'asymmetric',
 'asymmetric_alpha': 0.39792525383867455}

In [17]:
with open('/kaggle/working/Recsys_Challenge_2023/tuning_results/best_params_hybrid_merging.json', 'r') as params_file:
    params_merging = json.load(params_file)
params_merging

{'alpha': 0.3375591188839621, 'topK': 86}

In [18]:
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_train_validation.zip', 
    '/kaggle/working/ials_train_validation.zip', 
)
shutil.copyfile(
    '/kaggle/input/training-tuned-models/ials_all.zip', 
    '/kaggle/working/ials_all.zip', 
)

'/kaggle/working/ials_all.zip'

In [19]:
recommender_item = ItemKNNCFRecommender(URM_train)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_train)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_train)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_train)
recommender_p3a.fit(**params_p3alpha)

recommender_slim = SLIMElasticNetRecommender(URM_train)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_train_validation.zip')

recommender_ease_r = EASE_R_Recommender(URM_train)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_train_validation.zip')

recommender_ials = IALSRecommender(URM_train)
recommender_ials.load_model('/kaggle/working/', 'ials_train_validation.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_train, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
ItemKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 22347 (100.0%), 4283.62 column/sec. Elapsed time 5.22 sec
UserKNNCFRecommender: URM Detected 627 ( 4.8%) users with no interactions.
UserKNNCFRecommender: URM Detected 247 ( 1.1%) items with no interactions.
Similarity column 13024 (100.0%), 4269.13 column/sec. Elapsed time 3.05 sec
RP3betaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
RP3betaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2398.73 column/sec. Elapsed time 9.32 sec
P3alphaRecommender: URM Detected 627 ( 4.8%) users with no interactions.
P3alphaRecommender: URM Detected 247 ( 1.1%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2796.63 column/sec. Elapsed time 7.99 sec
SLIMElasticNetRecommender: URM Detected 627 ( 4.8%) users with n

In [20]:
evaluator = EvaluatorHoldout(URM_test_complete, cutoff_list=[10])

print('Merging Models')
result_df, _ = evaluator.evaluateRecommender(hybrid_similarity)
display(result_df)
print('Item KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_item)
display(result_df)
print('RP3 beta')
result_df, _ = evaluator.evaluateRecommender(recommender_rp3)
display(result_df)
print('P3 alpha')
result_df, _ = evaluator.evaluateRecommender(recommender_p3a)
display(result_df)
print('User KNN')
result_df, _ = evaluator.evaluateRecommender(recommender_user)
display(result_df)
print('SLIM Elastic')
result_df, _ = evaluator.evaluateRecommender(recommender_slim)
display(result_df)
print('EASE R')
result_df, _ = evaluator.evaluateRecommender(recommender_ease_r)
display(result_df)
print('IALS')
result_df, _ = evaluator.evaluateRecommender(recommender_ials)
display(result_df)

EvaluatorHoldout: Ignoring 2568 (19.7%) Users that have less than 1 test interactions
Merging Models
EvaluatorHoldout: Processed 10456 (100.0%) in 15.60 sec. Users per second: 670


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096595,0.165167,0.138965,0.050024,0.085384,0.274834,0.152525,0.11397,0.519702,0.365722,...,0.802826,0.41723,0.802826,0.083179,10.657938,0.998402,0.240004,0.821046,1.518592,0.309028


Item KNN
EvaluatorHoldout: Processed 10456 (100.0%) in 17.46 sec. Users per second: 599


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.09451,0.161146,0.135378,0.048719,0.082976,0.267977,0.148683,0.111312,0.511572,0.356662,...,0.802826,0.410703,0.802826,0.066309,10.283111,0.997789,0.191329,0.792171,1.734564,0.30426


RP3 beta
EvaluatorHoldout: Processed 10456 (100.0%) in 9.58 sec. Users per second: 1091


...

,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.097752,0.165757,0.139187,0.05124,0.086426,0.282607,0.154679,0.114847,0.52477,0.375707,...,0.802826,0.421299,0.802826,0.059374,10.474793,0.998673,0.171318,0.806937,1.547674,0.300111


EASE R
EvaluatorHoldout: Processed 10456 (100.0%) in 15.48 sec. Users per second: 676


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.087395,0.152428,0.129216,0.044693,0.077831,0.260324,0.140755,0.104268,0.497226,0.338086,...,0.802826,0.399186,0.802826,0.030673,9.45323,0.997032,0.088505,0.72824,2.076019,0.284205


IALS
EvaluatorHoldout: Processed 10456 (100.0%) in 15.92 sec. Users per second: 657


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.08224,0.144123,0.122645,0.041501,0.071584,0.246255,0.131701,0.098458,0.479055,0.316769,...,0.802826,0.384598,0.802826,0.029074,9.74689,0.998532,0.08389,0.750862,1.599865,0.289534


In [21]:
class PipelineRecommender(BaseRecommender):
    def __init__(self, URM_train, recommenders: list[BaseRecommender], cutoffs):
        super(PipelineRecommender, self).__init__(URM_train)
        self.recommenders = recommenders
        self.cutoffs = cutoffs
        
    def recommend(self, user_ids, cutoff=10, remove_seen_flag=True, remove_top_pop_flag=False, return_scores=True, remove_custom_items_flag=True):
        
        recommendations = None
        for recommender_cutoff, recommender in zip(self.cutoffs, self.recommenders):
            recommender_scores = recommender._compute_item_score(user_ids)
            
            for i, user_id in enumerate(user_ids):
                recommender_scores[i, :] = self._remove_seen_on_scores(user_id, recommender_scores[i, :])
            
            if recommendations is None:
                recommendations = np.argsort(recommender_scores, axis=1)[:, -recommender_cutoff:]
                
            else:
                filtered_scores = np.full_like(recommender_scores, -np.inf)
                for i in range(recommendations.shape[0]):
                    row_indices = recommendations[i]
                    filtered_scores[i, row_indices] = recommender_scores[i, row_indices]
                recommendations = np.argsort(filtered_scores, axis=1)[:, -recommender_cutoff:]              
                
        if return_scores:
            return np.flip(recommendations[:, -cutoff:], axis=1), filtered_scores
        return np.flip(recommendations[:, -cutoff:], axis=1)

In [22]:
pipeline_recommender = PipelineRecommender(
    URM_train, 
    recommenders=[recommender_p3a, hybrid_similarity, recommender_slim], 
    cutoffs=[500, 100, 10]
)

result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
display(result_df)

Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 28.94 sec. Users per second: 361


,PRECISION,PRECISION_RECALL_MIN_DEN,RECALL,MAP,MAP_MIN_DEN,MRR,NDCG,F1,HIT_RATE,ARHR_ALL_HITS,...,COVERAGE_USER,COVERAGE_USER_HIT,USERS_IN_GT,DIVERSITY_GINI,SHANNON_ENTROPY,RATIO_DIVERSITY_HERFINDAHL,RATIO_DIVERSITY_GINI,RATIO_SHANNON_ENTROPY,RATIO_AVERAGE_POPULARITY,RATIO_NOVELTY
cutoff,,,,,,,,,,,,,,,,,,,,,
10,0.096624,0.161291,0.134767,0.050992,0.085387,0.281697,0.15264,0.112552,0.519415,0.374128,...,0.802826,0.416999,0.802826,0.079937,10.86353,0.999015,0.230649,0.836884,1.249519,0.311412


In [23]:
recommender_objects_dict = {
    'merging': hybrid_similarity,
    'itemKnn': recommender_item,
    'userKnn': recommender_user,
    'rp3': recommender_rp3,
    'alpha': recommender_p3a,
    'ials': recommender_ials,
    'easeR': recommender_ease_r,
    'slim': recommender_slim
}

In [24]:
def objective(trial):
    recommenders_labels = list(recommender_objects_dict.keys())
    coarse_recommender = trial.suggest_categorical('coarse_recommender', recommenders_labels)
    use_3_levels = trial.suggest_categorical('use_3_levels', [True, False])
    
    if use_3_levels:
        recommenders_labels.remove(coarse_recommender)
        coarse_2_recommender = trial.suggest_categorical(
            f'coarse_2_recommender_c1:{coarse_recommender}', 
            recommenders_labels
        )
        recommenders_labels.remove(coarse_2_recommender)
        final_recommender = trial.suggest_categorical(
            f'final_recommender_c1:{coarse_recommender}_c2:{coarse_2_recommender}', 
            recommenders_labels
        )
        cutoff1 = trial.suggest_int('cutoff1', 200, 3000)
        cutoff2 = trial.suggest_int('cutoff2', 50, cutoff1)
        recommenders = [
            recommender_objects_dict[coarse_recommender], 
            recommender_objects_dict[coarse_2_recommender], 
            recommender_objects_dict[final_recommender]
        ]
        cutoffs = [cutoff1, cutoff2, 10]
        
    else:
        cutoff = trial.suggest_int('cutoff', 50, 3000)
        recommenders_labels.remove(coarse_recommender)
        final_recommender = trial.suggest_categorical(
            f'final_recommender_c1:{coarse_recommender}', 
            recommenders_labels
        )
        recommenders = [
            recommender_objects_dict[coarse_recommender], 
            recommender_objects_dict[final_recommender]
        ]
        cutoffs = [cutoff, 10]
    
    pipeline_recommender = PipelineRecommender(
        URM_train, 
        recommenders=recommenders, 
        cutoffs=cutoffs
    )

    result_df, _ = evaluator.evaluateRecommender(pipeline_recommender)
    return result_df.loc[10, 'MAP']

In [25]:
try:
    shutil.copyfile(
        '/kaggle/working/Recsys_Challenge_2023/tuning_results/tuning_hybrid.db', 
        config['database_path']
    )
except FileNotFoundError:
    pass # if not present optuna will create it

In [26]:
if config['tune_parameters']:
    study = optuna.create_study(direction='maximize', study_name='hyperparameters_tuning_variable_pipeline_v2', 
                                storage=f'sqlite:///{config["database_path"]}', load_if_exists=True)
    study.optimize(objective, n_trials=800)

[I 2023-12-08 22:33:17,726] A new study created in RDB with name: hyperparameters_tuning_variable_pipeline_v2


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 32.39 sec. Users per second: 323


[I 2023-12-08 22:33:50,360] Trial 0 finished with value: 0.04968313932305972 and parameters: {'coarse_recommender': 'alpha', 'use_3_levels': True, 'coarse_2_recommender_c1:alpha': 'easeR', 'final_recommender_c1:alpha_c2:easeR': 'merging', 'cutoff1': 1511, 'cutoff2': 1055}. Best is trial 0 with value: 0.04968313932305972.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 36.27 sec. Users per second: 288


[I 2023-12-08 22:34:26,836] Trial 1 finished with value: 0.03804894432907101 and parameters: {'coarse_recommender': 'easeR', 'use_3_levels': True, 'coarse_2_recommender_c1:easeR': 'rp3', 'final_recommender_c1:easeR_c2:rp3': 'userKnn', 'cutoff1': 250, 'cutoff2': 160}. Best is trial 0 with value: 0.04968313932305972.


...

[I 2023-12-09 04:28:46,978] Trial 797 finished with value: 0.048679933143878015 and parameters: {'coarse_recommender': 'easeR', 'use_3_levels': True, 'coarse_2_recommender_c1:easeR': 'slim', 'final_recommender_c1:easeR_c2:slim': 'itemKnn', 'cutoff1': 1589, 'cutoff2': 1146}. Best is trial 635 with value: 0.05134299361800376.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 27.63 sec. Users per second: 378


[I 2023-12-09 04:29:14,887] Trial 798 finished with value: 0.05133935402776317 and parameters: {'coarse_recommender': 'easeR', 'use_3_levels': False, 'cutoff': 204, 'final_recommender_c1:easeR': 'slim'}. Best is trial 635 with value: 0.05134299361800376.


Recommender_Base_Class: URM Detected 627 ( 4.8%) users with no interactions.
Recommender_Base_Class: URM Detected 247 ( 1.1%) items with no interactions.
EvaluatorHoldout: Processed 10456 (100.0%) in 27.26 sec. Users per second: 383


[I 2023-12-09 04:29:42,434] Trial 799 finished with value: 0.05133079207199388 and parameters: {'coarse_recommender': 'easeR', 'use_3_levels': False, 'cutoff': 142, 'final_recommender_c1:easeR': 'slim'}. Best is trial 635 with value: 0.05134299361800376.


In [27]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_optimization_history(study)
    fig.show()

In [28]:
if config['tune_parameters']:
    fig = optuna.visualization.plot_slice(study)
    fig.show()

In [29]:
if config['tune_parameters']:
    with open('/kaggle/working/best_params_pipeline_variable_length.json', 'w') as params_file:
        json.dump(study.best_params, params_file)
        
    if config['save_github']: 
        upload_file(
            '/kaggle/working/best_params_pipeline_variable_length.json', 
            'tuning_results/best_params_pipeline_variable_length.json', 
            'Hybrid Pipeline tuning results (from kaggle notebook)'
        )
    
elif config['copy_prev_best_params']:
    shutil.copyfile(
        '/kaggle/input/Recsys_Challenge_2023/tuning_results/best_params_pipeline_variable_length.json', 
        '/kaggle/working/best_params_pipeline_variable_length.json'
    )

In [30]:
if config['save_github'] and config['tune_parameters']:
    upload_file(
        config['database_path'], 
        'tuning_results/tuning_hybrid.db', 
        'Pipeline hybrid recommender tuning db updated results (from kaggle notebook)'
    )

# Training best model

In [31]:
recommender_item = ItemKNNCFRecommender(URM_all)
recommender_item.fit(**params_item_knn)

recommender_user = UserKNNCFRecommender(URM_all)
recommender_user.fit(**params_user_knn)

recommender_rp3 = RP3betaRecommender(URM_all)
recommender_rp3.fit(**params_rp3)

recommender_p3a = P3alphaRecommender(URM_all)
recommender_p3a.fit(**params_p3alpha)

recommender_slim = SLIMElasticNetRecommender(URM_all)
recommender_slim.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'slim_elastic_all.zip')

recommender_ease_r = EASE_R_Recommender(URM_all)
recommender_ease_r.load_model('/kaggle/working/Recsys_Challenge_2023/models/', 'ease_r_all.zip')

recommender_ials = IALSRecommender(URM_all)
recommender_ials.load_model('/kaggle/working/', 'ials_all.zip')

hybrid_similarity = ItemKNNSimilarityHybridRecommender(URM_all, recommender_item.W_sparse, recommender_rp3.W_sparse)
hybrid_similarity.fit(**params_merging)

ItemKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
ItemKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 22347 (100.0%), 4159.85 column/sec. Elapsed time 5.37 sec
UserKNNCFRecommender: URM Detected 386 ( 3.0%) users with no interactions.
UserKNNCFRecommender: URM Detected 125 ( 0.6%) items with no interactions.
Similarity column 13024 (100.0%), 3926.14 column/sec. Elapsed time 3.32 sec
RP3betaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
RP3betaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
RP3betaRecommender: Similarity column 22347 (100.0%), 2262.23 column/sec. Elapsed time 9.88 sec
P3alphaRecommender: URM Detected 386 ( 3.0%) users with no interactions.
P3alphaRecommender: URM Detected 125 ( 0.6%) items with no interactions.
P3alphaRecommender: Similarity column 22347 (100.0%), 2539.70 column/sec. Elapsed time 8.80 sec
SLIMElasticNetRecommender: URM Detected 386 ( 3.0%) users with n

In [32]:
recommender_objects_dict = {
    'merging': hybrid_similarity,
    'itemKnn': recommender_item,
    'userKnn': recommender_user,
    'rp3': recommender_rp3,
    'alpha': recommender_p3a,
    'ials': recommender_ials,
    'easeR': recommender_ease_r,
    'slim': recommender_slim
}

In [33]:
with open('/kaggle/working/best_params_pipeline_variable_length.json', 'r') as params_file:
    params = json.load(params_file)
    
if params['use_3_levels']:
    coarse1 = params['coarse_recommender']
    coarse2 = params[f'coarse_2_recommender_c1:{coarse1}']
    final = params[f'final_recommender_c1:{coarse1}_c2:{coarse2}']
    recommenders = [
        recommender_objects_dict[coarse1],
        recommender_objects_dict[coarse2], 
        recommender_objects_dict[final]
    ]
    cutoffs = [params['cutoff1'], params['cutoff2'], 10]
else:
    recommenders = [
        recommender_objects_dict[params['coarse_recommender']],
        recommender_objects_dict[params[f'final_recommender_c1:{params["coarse_recommender"]}']]
    ]
    cutoffs = [params['cutoff'], 10]
    
pipeline_recommender = PipelineRecommender(
    URM_all, 
    recommenders=recommenders,
    cutoffs=cutoffs
)

Recommender_Base_Class: URM Detected 386 ( 3.0%) users with no interactions.
Recommender_Base_Class: URM Detected 125 ( 0.6%) items with no interactions.


In [34]:
recommendations = pipeline_recommender.recommend(user_ids_test['user_id'].unique() - 1, cutoff=10, return_scores=False)
recommendations = [' '.join([str(item + 1) for item in r]).strip() for r in recommendations]

submission_df = pd.DataFrame({'user_id': user_ids_test['user_id'].unique(), 'item_list': recommendations})
display(submission_df.head())
submission_df.to_csv('/kaggle/working/submission.csv', index=False)

,user_id,item_list
0,1,101 36 123 506 515 403 694 1546 52 1422
1,2,1095 12 47 1522 50 656 949 359 54 102
2,3,59 857 2172 4252 1281 648 2748 956 259 536
3,4,249 28 50 314 171 146 139 254 187 128
4,5,1570 170 77 95 1511 471 131 5138 1220 135
